
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Associate/main/Includes/images/bookstore_schema.png" alt="Databricks Learning" style="width: 600">
</div>

In [0]:
%run ../Includes/Copy-Datasets


## Exploring The Source Directory

In [0]:
files = dbutils.fs.ls(f"{dataset_bookstore}/orders-raw")
display(files)

path,name,size,modificationTime
dbfs:/mnt/demo-datasets/bookstore/orders-raw/01.parquet,01.parquet,18823,1746436831000



## Auto Loader

In [0]:
(spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option("cloudFiles.schemaLocation", "dbfs:/mnt/demo/orders_checkpoint")
        .load(f"{dataset_bookstore}/orders-raw")
      .writeStream
        .option("checkpointLocation", "dbfs:/mnt/demo/orders_checkpoint")
        .table("orders_updates")
)

Out[10]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f06e5d9a7f0>

In [0]:
%sql
SELECT * FROM orders_updates LIMIT 3

order_id,order_timestamp,customer_id,quantity,total,books,_rescued_data
000000000006341,1657520256,C00788,1,41,"List(List(B08, 1, 41))",null
000000000006342,1657520256,C00788,1,41,"List(List(B08, 1, 41))",null
000000000006343,1657531717,C00654,1,28,"List(List(B02, 1, 28))",null


In [0]:
%sql
SELECT count(*) FROM orders_updates

count(1)
1000



## Landing New Files

In [0]:
# a function has been created before to load new file to this folder
load_new_data()

Loading 02.parquet file to the bookstore dataset


In [0]:
files = dbutils.fs.ls(f"{dataset_bookstore}/orders-raw")
display(files)

path,name,size,modificationTime
dbfs:/mnt/demo-datasets/bookstore/orders-raw/01.parquet,01.parquet,18823,1746436831000
dbfs:/mnt/demo-datasets/bookstore/orders-raw/02.parquet,02.parquet,18814,1746505605000


In [0]:
%sql
SELECT count(*) FROM orders_updates

count(1)
2000



## Exploring Table History

In [0]:
%sql
DESCRIBE HISTORY orders_updates

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-05-06T04:26:47.000+0000,983714032662280,luubaonluu@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> de767cfc-5b0c-4555-93f0-fa580a849046, epochId -> 1)",null,List(763081737063962),0506-021029-b15wszst,1,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 1000, numOutputBytes -> 19051, numAddedFiles -> 1)",null,Databricks-Runtime/12.2.x-scala2.12
1,2025-05-06T04:24:56.000+0000,983714032662280,luubaonluu@gmail.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> de767cfc-5b0c-4555-93f0-fa580a849046, epochId -> 0)",null,List(763081737063962),0506-021029-b15wszst,0,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 1000, numOutputBytes -> 19060, numAddedFiles -> 1)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-05-06T04:24:49.000+0000,983714032662280,luubaonluu@gmail.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(763081737063962),0506-021029-b15wszst,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12



## Cleaning Up

In [0]:
%sql
DROP TABLE orders_updates

In [0]:
dbutils.fs.rm("dbfs:/mnt/demo/orders_checkpoint", True)